# Lab 9
## Steven Hunt / Will Sparks
### 11/5/19

Import Python libraries

In [1]:
import pymongo
from pymongo import MongoClient
import json
import tweepy
import twitter
from pprint import pprint
import configparser

### Load the Authorization Info
Save all the secretes or tokens in a config.ini file and use the configparse to load the authorization info.

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')

CONSUMER_KEY      = config['mytwitter']['api_key']
CONSUMER_SECRET   = config['mytwitter']['api_secrete']
OAUTH_TOKEN       = config['mytwitter']['access_token']
OATH_TOKEN_SECRET = config['mytwitter']['access_secrete']

mongod_connect = config['mymongo']['connection']

### Connect to the MongoDB Cluster

In [3]:
client = MongoClient(mongod_connect)
db = client.tweet_db # create a database named tweet_db
tweet_collection = db.tweet_collection_job #create a collection called tweet_collection
tweet_collection.create_index([("id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'id_1'

### Use the Streaming API to Collect Tweets
Authorize the Stream API

In [4]:
stream_auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
stream_auth.set_access_token(OAUTH_TOKEN, OATH_TOKEN_SECRET)

strem_api = tweepy.API(stream_auth)

Define the query for the Stream API

In [13]:
track = ["intelligence analyst job"] # define the keywords, tweets about intelligence

The collected tweets will contain 'intelligence analyst'

In [14]:
class MyStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        print (status.id_str)
        try:
            tweet_collection.insert_one(status._json)
        except:
            pass
  
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_data disconnects the stream
            return False
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = strem_api.auth, listener=myStreamListener)
myStream.filter(track=track)

### Use the REST API to Collect Tweets
Authorize the REST API

In [7]:
rest_auth = twitter.oauth.OAuth(OAUTH_TOKEN,OATH_TOKEN_SECRET,CONSUMER_KEY,CONSUMER_SECRET)
rest_api = twitter.Twitter(auth=rest_auth)

Define the query for the REST API

In [8]:
count = 100 #number of returned tweets, default and max is 100
q = ["intelligence analyst job"]                               #define the keywords, tweets about IA jobs

The collected tweets will contain 'JMU' AND are located in Harrisonburg, VA

In [16]:
search_results = rest_api.search.tweets( count=count,q=q)
statuses = search_results["statuses"]
since_id_new = statuses[-1]['id']
for statuse in statuses:
    try:
        tweet_collection.insert_one(statuse)
        pprint(statuse['created_at'])# print the date of the collected tweets
    except:
        pass

Continue fetching early tweets with the same query.

YOU WILL REACH YOUR RATE LIMIT VERY FAST

In [17]:
since_id_old = 0
while(since_id_new != since_id_old):
    since_id_old = since_id_new
    search_results = rest_api.search.tweets( count=count,q=q,
                        max_id= since_id_new)
    statuses = search_results["statuses"]
    since_id_new = statuses[-1]['id']
    for statuse in statuses:
        try:
            tweet_collection.insert_one(statuse)
            pprint(statuse['created_at']) # print the date of the collected tweets
        except:
            pass

'Sun Nov 03 14:56:31 +0000 2019'
'Sun Nov 03 14:41:47 +0000 2019'
'Sun Nov 03 14:32:32 +0000 2019'
'Sun Nov 03 14:14:16 +0000 2019'
'Sun Nov 03 14:01:27 +0000 2019'
'Sun Nov 03 13:50:04 +0000 2019'
'Sun Nov 03 13:33:57 +0000 2019'
'Sun Nov 03 13:25:48 +0000 2019'
'Sun Nov 03 13:22:42 +0000 2019'
'Sun Nov 03 13:20:13 +0000 2019'
'Sun Nov 03 13:18:07 +0000 2019'
'Sun Nov 03 13:12:52 +0000 2019'
'Sun Nov 03 13:09:13 +0000 2019'
'Sun Nov 03 12:36:04 +0000 2019'
'Sun Nov 03 12:32:04 +0000 2019'
'Sun Nov 03 12:24:36 +0000 2019'
'Sun Nov 03 12:18:03 +0000 2019'
'Sun Nov 03 12:11:01 +0000 2019'
'Sun Nov 03 11:49:20 +0000 2019'
'Sun Nov 03 09:25:13 +0000 2019'
'Sun Nov 03 09:24:27 +0000 2019'
'Sun Nov 03 08:41:15 +0000 2019'
'Sun Nov 03 08:11:22 +0000 2019'
'Sun Nov 03 07:56:10 +0000 2019'
'Sun Nov 03 07:44:28 +0000 2019'
'Sun Nov 03 06:26:23 +0000 2019'
'Sat Nov 02 23:21:59 +0000 2019'
'Sat Nov 02 22:08:48 +0000 2019'
'Sat Nov 02 21:33:15 +0000 2019'
'Sat Nov 02 21:09:52 +0000 2019'
'Sat Nov 0

### Print the Collected Tweets

In [18]:
print(tweet_collection.estimated_document_count())# number of tweets collected

user_cursor = tweet_collection.distinct("user.id")
print (len(user_cursor)) # number of unique Twitter users

454
373


Create a text index and print the Tweets containing specific keywords

In [19]:
tweet_collection.create_index([("text", pymongo.TEXT)], name='text_index', default_language='english') # create a text index

tweet_cursor = tweet_collection.find({"$text": {"$search": "intelligence analyst job"}}) # return tweets about IA jobs
for document in tweet_cursor:
    try:
        print ('----')
#         pprint (document) # use pprint to print the entire tweet document
   
        print ('name:', document["user"]["name"]) # user name
        print ('text:', document["text"])         # tweets
    except:
        print ("***error in encoding")
        pass

----
name: TMJ-OHD IT Jobs
text: Have you ever had a role like "MENA Language Intelligence Analyst"? What did you like most about that job? #BoozAllen #Analyst
----
name: TMJ-OHD IT Jobs
text: We're hiring! Click to apply: Space Intelligence Analyst, Mid - https://t.co/YypeutbdxS #BoozAllen #Analyst
----
name: King's Lynn Live
text: RT @Kings_Lynn_Jobs: #KingsLynnJobs Trainee Intelligence Analyst, Nationwide: Trainee Intelligence Analyst YOUR ARMY NEEDS YOU. Naturally c…
----
name: Winchester, VA Aero.
text: We're hiring! Read about our latest job opening here: Intelligence Threat Analyst - https://t.co/SPjEzm7v6U #BoozAllen #Analyst
----
name: Leatherhead Jobs
text: #LeatherheadJobs Leatherhead Jobs: Trainee Intelligence Analyst, Nationwide: Trainee Intelligence Analyst YOUR ARMY… https://t.co/J0OABE8HoU
----
name: Kings Lynn Jobs
text: #KingsLynnJobs Trainee Intelligence Analyst, Nationwide: Trainee Intelligence Analyst YOUR ARMY NEEDS YOU. Naturall… https://t.co/DBdLMt9Pwh
----
name

text: Join the AT&amp;T team! See our latest #Database job openings, including "Senior Business Analyst, Data Management &amp; Bu… https://t.co/iT98VcBTW9
----
name: Jobs Italia
text: Accenture: Market intelligence An... https://t.co/LFpAr5aVu5
----
name: Jobs New Zealand
text: NZ Customs: Senior Intelligence A... https://t.co/bo5Zm3iwnT
----
name: TMJ-OK Govern Jobs
text: There is more to SAIC than partnering with the intelligence community. We understand the importance of partnering w… https://t.co/cTvhAHgh3Y
----
name: TMJ-BAL IT PM Jobs
text: There is more to SAIC than partnering with the intelligence community. We understand the importance of partnering w… https://t.co/bpkxdl7G25
----
name: LEGO_Careers
text: Are you interested in working for a fast paced, change-oriented organization? Apply to our Business Intelligence An… https://t.co/mYGA7VC4HA
----
name: TMJ-AFG Jobs
text: If you're looking for work in Kabul, Afghanistan, check out this Intelligence job via the link in our bio